<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.3 Data Access and Basic Processing

## Filter Operators

When interacting with large data collections, it is necessary to keep in mind that it's not possible to load everything!

Therefore, we always have to define our requirements in advance and apply them to the data using filter operators.

Let's start again with the same sample data from the Sentinel-2 STAC Collection with an additional filter.

### Properties Filter

When working with optical data like Sentinel-2, most of the times we would like to discard cloudy acquisitions as soon as possible.

We can do it using a property filter: in this case we want to keep only the acquisitions with less than 50% cloud coverage.

In [1]:
properties = {"eo:cloud_cover": dict(lt=50)}

In [2]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"
spatial_extent = {"west": 11.1, "east": 11.5, "south": 46.1, "north": 46.5}

datacube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent,
                    properties=properties)
datacube.execute()

Did not load machine learning processes due to missing dependencies: Install them like this: `pip install openeo-processes-dask[implementations, ml]`
/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-e9f6fc9f4f9a2a10d139d92d785674fb' (time: 619,
                                                                band: 32,
                                                                y: 4534, x: 3209)>
dask.array<getitem, shape=(619, 32, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2016-11-08...
    id                                       (time) <U24 'S2A_32TPS_20161108_...
  * band                                     (band) <U12 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 6.611e+05 ... 6.932e+05
  * y                                        (y) float64 5.153e+06 ... 5.107e+06
    mgrs:latitude_band                       <U1 'T'
    ...                                       ...
    raster:bands                             (band) object [{'nodata': 0, 'da...
    gsd                                      (band) object None 10 ... None None
    common_name                              (band) object None 'blue' ... None
    center_wavelength                        (band) object None 0.49 ... None
    full_width_half_max                      (band) object None 0.098 ... None
    epsg                                     int64 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

### Temporal filter

To slice along time the data collection with openEO, we can use the `filter_temporal` process.

In [3]:
temporal_extent = ["2022-05-10T00:00:00Z","2022-06-30T00:00:00Z"]
temporal_slice = datacube.filter_temporal(temporal_extent)
temporal_slice.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-2cafb4a8082478f674b2548714d041bf' (time: 9,
                                                                band: 32,
                                                                y: 4534, x: 3209)>
dask.array<getitem, shape=(9, 32, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2022-05-11...
    id                                       (time) <U24 'S2A_32TPS_20220511_...
  * band                                     (band) <U12 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 6.611e+05 ... 6.932e+05
  * y                                        (y) float64 5.153e+06 ... 5.107e+06
    mgrs:latitude_band                       <U1 'T'
    ...                                       ...
    raster:bands                             (band) object [{'nodata': 0, 'da...
    gsd                                      (band) object None 10 ... None None
    common_name                              (band) object None 'blue' ... None
    center_wavelength                        (band) object None 0.49 ... None
    full_width_half_max                      (band) object None 0.098 ... None
    epsg                                     int64 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

After running the previous cell, it is visible that the result has less elements (or labels) in the temporal dimension `time`.

Additionally, the size of the selected data reduced a lot.

**Quiz hint: look carefully at the dimensions of the resulting datacube!**

### Spatial filter

To slice along the spatial dimensions the data collection with openEO, we can use `filter_bbox` or `filter_spatial` processes.

The `filter_bbox` process is used with a set of coordinates:

In [4]:
spatial_extent = {"west": 11.259613, "east": 11.406212, "south": 46.461019, "north": 46.522237}
spatial_slice = datacube.filter_bbox(spatial_extent)
spatial_slice.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-500411841153698219b5cec0ed46d725' (time: 619,
                                                                band: 32,
                                                                y: 489, x: 1145)>
dask.array<getitem, shape=(619, 32, 489, 1145), dtype=float64, chunksize=(1, 1, 382, 860), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2016-11-08...
    id                                       (time) <U24 'S2A_32TPS_20161108_...
  * band                                     (band) <U12 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 6.733e+05 ... 6.848e+05
  * y                                        (y) float64 5.153e+06 ... 5.148e+06
    mgrs:latitude_band                       <U1 'T'
    ...                                       ...
    raster:bands                             (band) object [{'nodata': 0, 'da...
    gsd                                      (band) object None 10 ... None None
    common_name                              (band) object None 'blue' ... None
    center_wavelength                        (band) object None 0.49 ... None
    full_width_half_max                      (band) object None 0.098 ... None
    epsg                                     int64 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

**Quiz hint: look carefully at the dimensions of the loaded datacube!**

### Bands filter

To slice along the bands dimension, keeping only the necessary bands, we can use the `filter_bands` process.

In [6]:
bands = ["red","green","blue"]
bands_slice = datacube.filter_bands(bands)
bands_slice.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-9e961c20df1c0bb69e386cae970a2276' (time: 619,
                                                                band: 3,
                                                                y: 4534, x: 3209)>
dask.array<getitem, shape=(619, 3, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2016-11-08...
    id                                       (time) <U24 'S2A_32TPS_20161108_...
  * band                                     (band) <U12 'red' 'green' 'blue'
  * x                                        (x) float64 6.611e+05 ... 6.932e+05
  * y                                        (y) float64 5.153e+06 ... 5.107e+06
    mgrs:latitude_band                       <U1 'T'
    ...                                       ...
    raster:bands                             (band) object None None None
    gsd                                      (band) object 10 10 10
    common_name                              (band) object 'red' 'green' 'blue'
    center_wavelength                        (band) object 0.665 0.56 0.49
    full_width_half_max                      (band) object 0.038 0.045 0.098
    epsg                                     int64 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0